In [9]:
import pandas as pd
import boto3
import re
import tempfile

## Transcribe

### Call Processed Dataset

In [12]:
df = pd.read_csv('merged_crm_meta.csv')
df['Initiation timestamp_yyyymm'] = df['Initiation timestamp'].apply(lambda x: str(x)[:4] + str(x)[5:7])
df['Initiation timestamp_yyyymmdd'] = df['Initiation timestamp'].apply(lambda x: str(x)[:4] + str(x)[5:7] + str(x)[8:10])

### List Calls

In [11]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('fwd-sg-sagemaker-raw-data')
list_of_wav_to_transcribe = []

for obj in bucket.objects.filter(Prefix = 'call_center/CallRecordings/2021/08/'):
    list_of_wav_to_transcribe.append(obj.key)

list_of_wav_to_transcribe = list_of_wav_to_transcribe[1:] # Pops the first entry, which is only prefix and no .wav file
print('Number of wav files to transcribe: {}'.format(len(list_of_wav_to_transcribe)))

Number of wav files to transcribe: 12117


In [13]:
sample_contact_ids = df['Contact ID'].head(4).values
sample_initiation_timestamps = ['13','13','13','13'] # Day of August
s3 = boto3.resource('s3')
s3c = boto3.client('s3')
bucket = s3.Bucket('fwd-sg-sagemaker-raw-data')

for contact_id, day in zip(sample_contact_ids, sample_initiation_timestamps):
    for obj in bucket.objects.filter(Prefix = 'call_center/CallRecordings/2021/08/{}'.format(day)):
        if re.search(contact_id, obj.key):
            print(obj.key)

In [10]:
transcribe = boto3.client('transcribe')
jobs = [] # This is a list of jobs that are currently running

for index, wav in enumerate(list_of_wav_to_transcribe[:2]):
    
    if index % 200 == 0:
        # Every 200 calls wait for transcriptions to complete (because limit is 250)
        while len(jobs) > 5:
            for job in jobs:
                # Sleep to avoid reaching API limit
                time.sleep(0.05)
                # Check job status for when its done, logging when jobs are done
                status = transcribe.get_transcription_job(TranscriptionJobName=job)
                if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                    jobs.remove(job)
                    print(job + ' is done, ' + str(len(jobs)) + ' jobs left to be completed.')
            
            time.sleep(30)
        
    # Record down info about this wav
    prefix1, prefix2, year, month, day, contact_id = wav.split('/')
    job_name = 'August-transcription-{}'.format(contact_id)
    job_uri = 'fwd-sg-sagemaker-raw-data/' + wav  # s3 uri of the wav file
    
    transcribe.start_transcription_job(
                TranscriptionJobName = job_name,
                Media={'MediaFileUri': job_uri},
                MediaFormat='wav',
                OutputBucketName='fwd-sg-sagemaker-raw-data',
                OutputKey='call_center/Transcribe_output/{}/{}/{}/{}'.format(year, month, day, contact_id),
                LanguageCode='en-US',
                Settings={
                    'VocabularyName': 'list-vocab-basic',
                    'ChannelIdentification': True,
                    'ShowAlternatives': True,
                    'MaxAlternatives': 2
                })
    
    jobs.append(job_name)
    time.sleep(0.05)

ClientError: An error occurred (AccessDeniedException) when calling the StartTranscriptionJob operation: User: arn:aws:sts::365339902192:assumed-role/Sagemaker-Access-Role/SageMaker is not authorized to perform: transcribe:StartTranscriptionJob on resource: arn:aws:transcribe:ap-southeast-1:365339902192:vocabulary/list-vocab-basic because no identity-based policy allows the transcribe:StartTranscriptionJob action

In [ ]:
transcribe = boto3.client('transcribe')
jobs = [] # This is a list of jobs that are currently running

for index, wav in enumerate(list_of_wav_to_transcribe[:2]):
    
    if index % 200 == 0:
        # Every 200 calls wait for transcriptions to complete (because limit is 250)
        while len(jobs) > 5:
            for job in jobs:
                # Sleep to avoid reaching API limit
                time.sleep(0.05)
                # Check job status for when its done, logging when jobs are done
                status = transcribe.get_transcription_job(TranscriptionJobName=job)
                if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                    jobs.remove(job)
                    print(job + ' is done, ' + str(len(jobs)) + ' jobs left to be completed.')
            
            time.sleep(30)
        
    # Record down info about this wav
    prefix1, prefix2, year, month, day, contact_id = wav.split('/')
    job_name = 'August-transcription-{}'.format(contact_id)
    job_uri = 'fwd-sg-sagemaker-raw-data/' + wav  # s3 uri of the wav file
    
    transcribe.start_transcription_job(
                TranscriptionJobName = job_name,
                Media={'MediaFileUri': job_uri},
                MediaFormat='wav',
                OutputBucketName='fwd-sg-sagemaker-raw-data',
                OutputKey='call_center/Transcribe_output/{}/{}/{}/{}'.format(year, month, day, contact_id),
                LanguageCode='en-US',
                Settings={
                    'VocabularyName': 'list-vocab-basic',
                    'ChannelIdentification': True,
                    'ShowAlternatives': True,
                    'MaxAlternatives': 2
                })
    
    jobs.append(job_name)
    time.sleep(0.05)

In [ ]:
%%time
from __future__ import print_function
import time
import boto3

# Programmatic Transcription
transcribe = boto3.client('transcribe')
jobs = []


# For every call to be transcribed
for i in range(len(to_trans)):
    
    # Every 200 calls wait for transcriptions to complete (because limit is 250)
    if (i % 200) is 0:
        print('\n\n Waiting Loop...')
        # Only allow 5 jobs to be transcribing before continuing
        while len(jobs) > 5:
            for job in jobs:
                # Sleep to avoid reaching API limit
                time.sleep(0.05)
                # Check job status for when its done, logging when jobs are done
                status = transcribe.get_transcription_job(TranscriptionJobName=job)
                if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                    jobs.remove(job)
                    print(job + ' is done, ' + str(len(jobs)) + ' jobs left to be completed.')
            
            time.sleep(30)
            
    try:
        # THESE CAN BE CHANGED
        job_name = 'August-transcribe' + file_name.replace('%', '')
        
        # Make sure these are correct
        file_name = to_trans[i][39:]
        job_uri = "s3://singapore-polytechnic-voice-to-text/" + to_trans[i]
        print(job_uri)
        
        # SET TRANSCRIPTION PARAMETERS HERE
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat='wav',
            OutputBucketName='singapore-polytechnic-voice-to-text',
            OutputKey='jan_2021_transcripts/',
            LanguageCode='en-US',
            Settings={
                'VocabularyName': 'list-vocab-basic',
                'ChannelIdentification': True,
                'ShowAlternatives': True,
                'MaxAlternatives': 2
            },
        )

        print('Created job ' + job_name)
        jobs.append(job_name)
        # Avoid spamming API
        time.sleep(0.05)
    except:
        pass

# For last batch of transcriptions
while True:
    for job in jobs:
        time.sleep(0.05)
        status = transcribe.get_transcription_job(TranscriptionJobName=job)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            jobs.remove(job)
            print(job + ' is done, ' + str(len(jobs)) + ' jobs left to be completed.')
            
    if len(jobs) == 0:
        break
    time.sleep(30)
print('\n\n\n\nDONE!')